# Exploratory data analysis

**Inputs:**

- Configuration file
- Raw data

**Steps:**

- Load configuration
- Load raw data
- Create summary statistics
- Create plots

**Outputs:**

- Summary statistics
- Plots